In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
os.listdir('../input')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Simple parse of the 'games.json' file.
import os
import json

dataset = {}
if os.path.exists('/kaggle/input/steam-games-dataset/games.csv'):
    with open('/kaggle/input/steam-games-dataset/games.csv', 'r', encoding='utf-8') as fin:
        text = fin.read()
        if len(text) > 0:
            df = pd.read_csv("/kaggle/input/steam-games-dataset/games.csv")
else:
    print("file not found")

In [ ]:
from datetime import datetime, timedelta

#Util. functions for later
def getBoxleiterRatio(_date):
    year = _date.year
    index = year - 2013
    # 2013 and before, 2014, 2015, 2016, 2017, 2018, 2019, 2020 and onwards
    ratios = [60,50,50,50,40,35,35,30]
    index = min(max(0,index),len(ratios)-1)
    return ratios[index]
    
    
def estimatedSales(_review_count,_date):
    return _review_count * getBoxleiterRatio(_date)

#Test
#print(getBoxleiterRatio(datetime(2015,1,1)))

In [ ]:
df["Review Count"]=df['Positive']+df['Negative']
#Treat release date to be used with datetime
df['Release date'] = pd.to_datetime(df['Release date'])  

df['Estimated Sales'] = df.apply(lambda x: estimatedSales(x['Review Count'],x['Release date']),axis=1)

df['Estimated Rev'] = df.apply(lambda x: x['Estimated Sales'] * x['Price'],axis=1)

print("Before filtering: "+ str(len(df)))

# Filter out free games
df = df[df["Price"]>0.0]
print("After free game filtering: "+ str(len(df)))

#Filter out games released before Steam Direct
df = df[(df['Release date'] >= pd.to_datetime('2017-06-13'))]
print("After Steam Direct filtering: "+ str(len(df)))

#Filter out games with less than 10 reviews, due to how Steam works
df = df[df["Review Count"]>=10]
print("After review count filter: " + str(len(df)))

#Filter out games released in less than 30 days. Most of the revenue is made in the first month of release, so revenue estimate would be very innacurate.
df = df[df["Release date"] < datetime.today() - timedelta(days= 30)]
print("After month filter: " + str(len(df)))

# Filter out top X% games by reveneue
df = df.sort_values(by='Estimated Rev',ascending=True).head( int(round((len(df)+1)*(98/100))) )
print("After top percentile filter: " + str(len(df)))

# Filter out games priced less than 1 USD, as they are probably not relevant
df = df[df["Price"]>=1.0]
print("After price filter: " + str(len(df)))

# Filter out games by estimated reveneue
# df = df[df["Estimated Rev"] >= 50000]
# print("After Rev filter: " + str(len(df)))

#print("Before percentile filter: " + str(len(df)))
## Filter out bottom X% games by reveneue
#df = df.sort_values(by='Estimated Rev',ascending=False).head( int(round((len(df)+1)*(97/100))) )

print("After all filtering: " + str(len(df)))

In [ ]:
# String to list
df['Tags'] = df.apply(lambda x: x['Tags'].split(','),axis=1)
df['Developers'] = df.apply(lambda x: x['Developers'].split(','),axis=1)
df['Publishers'] = df['Publishers'].apply(lambda x: str(x) if isinstance(x, float) else x) # Convert float to text
df['Publishers'] = df.apply(lambda x: x['Publishers'].split(','),axis=1)

In [ ]:
def to_1D(series):
    #Sauce: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
    return pd.Series([x for _list in series for x in _list])

tag_frequency = to_1D(df["Tags"]).value_counts()

In [ ]:
def boolean_df(item_lists, unique_items):
# Sauce: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

df_bool = boolean_df(df["Tags"], tag_frequency.keys())

df_bool['Estimated Sales'] = df['Estimated Sales']
df_bool['Estimated Rev'] = df['Estimated Rev']
df_bool['Name'] = df['Name']
df_bool['AppID'] = df['AppID']
df_bool['Price'] = df['Price']
df_bool['Release Date'] = df['Release date']

# Indie-only filter
# print("Before indie filter: " + str(len(df_bool)))
# df_bool = df_bool[(df_bool['Indie']==True)]
# print("After indie filter: " + str(len(df_bool)))

In [ ]:
df_tag = pd.DataFrame()
df_tag['Count'] = tag_frequency
df_tag.set_index('Count')
df_tag['Median Rev'] = 0

for key in tag_frequency.keys():
    if len(df_bool.loc[df_bool[key] == True]) > 0:
        df_tag.at[key,'Median Sales'] = df_bool.loc[df_bool[key] == True, 'Estimated Sales'].median()
        df_tag.at[key,'Median Rev'] = df_bool.loc[df_bool[key] == True, 'Estimated Rev'].median()
        df_tag.at[key,'Quartile'] = df_bool.loc[df_bool[key] == True, 'Estimated Rev'].quantile(0.25)
        df_tag.at[key,'Median Price'] = ( df_bool.loc[df_bool[key] == True, 'Price'].median() )

In [ ]:
# I consider these tags not as relevant for indie games - either because it requires licensing,is out of control of the developer (eg.: classic), is not about games, or is not really a genre/theme.
# Also Musou games are too much of an outlier, most if not all are made by the same company
# This filtering doesn't affect the stats, just what is displayed in the graphs
tags_exclude = ['Musou','Warhammer 40K', 'LEGO','Games Workshop','Crowdfunded','Batman','Coding','Indie','Steam Machine','Feature Film','Instrumental Music','Reboot','Remake','Benchmark','Video Editting','Masterpiece','360 Video','Jump Scare','Documentary','8-bit Music','Hardware','TrackIR','Great Soundtrack','Cult Classic','Classic','Kickstarter','Mod','Quick-time Events','Addictive','Short','Tutorial',"Gaming","Free to Play","Software","Audio Production","Software Training","Utilities","4 Player Local","Quick-Time Events","Snow","Based On A Novel","e-sports","Foreign",'Colorful','Photo Editing','Silent Protagonist',"1990's",'Design & Illustration','Artificial Intelligence','Programming','Animation & Modeling','Blood','Early Access','Web Publishing','Nudity','Epic','Soundtrack','Movie','GameMaker','Video Production','Sequel','Video Editting']
df_tag = df_tag.drop(tags_exclude,errors='ignore')

# Other tags that are arguably genres but that aren't that useful for my personal purposes because I'm unable to do them justice or that I'm not interested. Remove or edit this if you plan to use this notebook
tags_exclude = tags_exclude + ['MMORPG','Multiplayer','Sexual Content','Hentai','World War I','World War II','Tennis','Motocross','Motorbike','Skating','Racing','Nostalgia','Fishing','Offroad','Politics','Political Sim','Rome','Cold War','Mature','Volleyball','Naval','Episodic','Trains','Submarine','Illuminati','Voice Control','Moddable','Naval Combat','Wrestling','Sailing','Cycling','NSFW','Cricket','Bowling','BMX','Snowboarding','Mars','Skiing','Hockey','Rugby','LGBTQ+','Bikes','Pool','Asymmetric VR','VR','Driving','Automobile Sim','Vehicular Combat','Political','Beautiful','Faith','Skateboarding','FMV','Science','Dog','Horse','Jet','Satire','Gun Customization','Comic Book','Mini Golf','1980s','Modern','Level Editor','Demons','Dragons','Violent','Gore','Vikings','Horses']
#Over-populated generic tags that aren't super helpful
tags_exclude = tags_exclude + ['Action','Adventure','2D','Casual','Singleplayer']
df_tag = df_tag.drop(tags_exclude,errors='ignore')

#print( df_tag.sort_values(['Median Rev','Count'],ascending=[False,True]).head(n=50) )

In [ ]:
import matplotlib
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

#EDIT ME HERE: Genre to analyse, this will change the result of one of the cells below and the next scatter plot colors
genre = "Anime"
genre2 = None

#Necessary to be able to run the cell without executing everything again
df_tag = df_tag.drop('CUSTOM',axis=0,errors='ignore')
df_tag = df_tag.drop('Color',axis=1,errors='ignore')
df_tag = df_tag.drop('Size',axis=1,errors='ignore')

if genre2 and genre2 in df_tag.keys():
    df_tag.loc['CUSTOM']=[None,None,None,None]
    df_tag.at['CUSTOM','Count'] = len(df_bool[ (df_bool[genre] == True) & (df_bool[genre2] == True)])
    df_tag.at['CUSTOM','Median Rev'] = df_bool.loc[ (df_bool[genre] == True) & (df_bool[genre2] == True), 'Estimated Rev'].median()
    df_tag.at['CUSTOM','Quartile'] = df_bool.loc[ (df_bool[genre] == True) & (df_bool[genre2] == True), 'Estimated Rev'].quartile(0.25)
    df_tag.at['CUSTOM','Median Sales'] = df_bool.loc[ (df_bool[genre] == True) & (df_bool[genre2] == True), 'Estimated Sales'].median()
    df_tag.at['CUSTOM','Median Price'] = df_bool.loc[ (df_bool[genre] == True) & (df_bool[genre2] == True), 'Price'].median()
    if df_tag.at['CUSTOM','Count'] == 0:
        print("No tag combination found!")

#Size of the plot
plt.rcParams['figure.figsize'] = [75, 75]

fig,ax = plt.subplots()
df_tag['Color'] = "#99d7e8"
df_tag['Size'] = 200.0
df_tag.at[genre,'Color']= '#e80042'
if genre2:
    df_tag.at[genre2,'Color']= '#e80042'
    df_tag.at['CUSTOM','Color']= '#d820f8'
df_tag.at[genre,'Size']= 450.0
if genre2:
    df_tag.at[genre2,'Size']= 450.0
    df_tag.at['CUSTOM','Size']= 450.0
ax.axvline(x=0,c="red")
ax.axhline(y=0,c="red")

ax.axhline(y=50000)
ax.axvline(x=1250)

first_axis = "Count"
second_axis = "Median Rev"
third_axis = "Quartile"

plt.xlabel(first_axis,fontdict={'family':'serif','color':'black','size':25})
plt.ylabel(second_axis,fontdict={'family':'serif','color':'black','size':25})
ax.scatter(df_tag[first_axis].to_list(), df_tag[second_axis].to_list(),c=df_tag['Color'].to_list(),s=(df_tag[third_axis]/20).to_list())


for i, txt in enumerate(df_tag.index):
    text = ax.annotate(txt, (df_tag[first_axis].to_list()[i], df_tag[second_axis].to_list()[i]),rotation=30,size= 20,ha="left",rotation_mode="anchor")
    text.set_alpha(.5)

In [ ]:
#Sub genre analysis (matches genre specified above)

df_tag2 = pd.DataFrame()
df_tag2['Median Rev'] = 0
for key in tag_frequency.keys():
    if len(df_bool[df_bool[key] & df_bool[genre]]) > 0:
        df_tag2.at[key,'Count'] = len(df_bool[df_bool[key] & df_bool[genre]])
        df_tag2.at[key,'Median Rev'] = df_bool.loc[df_bool[key] & df_bool[genre], 'Estimated Rev'].median()
        df_tag2.at[key,'Quartile'] = df_bool.loc[df_bool[key] & df_bool[genre], 'Estimated Rev'].quantile(0.25)
        df_tag2.at[key,'Median Sales'] = df_bool.loc[df_bool[key] & df_bool[genre], 'Estimated Sales'].median()
        df_tag2.at[key,'Median Price'] = ( df_bool.loc[df_bool[key] & df_bool[genre], 'Price'].median() )
df_tag2 = df_tag2.drop(tags_exclude,errors='ignore')       

#Necessary to be able to run the cell without executing everything again
df_tag2 = df_tag2.drop('Color',axis=1,errors='ignore')
df_tag2 = df_tag2.drop('Size',axis=1,errors='ignore')

#Size of the plot
plt.rcParams['figure.figsize'] = [75, 75]

genre_dot_size_multiplier = 1.5

fig,ax = plt.subplots()
df_tag2['Color'] = "#99d7e8"
df_tag2['Size'] = 200.0
df_tag2.at[genre,'Color']= '#e80042'
if genre2:
    df_tag2.at[genre2,'Color']= '#e80042'
df_tag2.at[genre,'Size']= df_tag2.at[genre,'Size']*genre_dot_size_multiplier
if genre2:
    df_tag2.at[genre2,'Size']= df_tag2.at[genre2,'Size']*genre_dot_size_multiplier
    
ax.axhline(y=df_tag2.loc[genre,second_axis],c="black",ls='--')

ax.scatter(df_tag2[first_axis].to_list(), df_tag2[second_axis].to_list(),c=df_tag2['Color'].to_list(),s=(df_tag2['Count']/5).to_list())

for i, txt in enumerate(df_tag2.index):
    text = ax.annotate(txt, (df_tag2[first_axis].to_list()[i], df_tag2[second_axis].to_list()[i]),rotation=30,size= 12,ha="right",rotation_mode="anchor")
    text.set_alpha(0.5)

In [ ]:
df_association = df_bool.drop(tags_exclude+['Estimated Sales','Estimated Rev','Name','AppID','Price','Quartile'],axis=0,errors='ignore').drop(tags_exclude+['Estimated Sales','Estimated Rev','Name','AppID','Price'],axis=1,errors='ignore').corr(method="pearson")

#import seaborn as sn
#plt.ioff() #Disable interactive mode to not output a bunch of text
#fig, ax = plt.subplots(figsize = (60,60))

##Color map
#top = cm.get_cmap('Oranges_r', 128)
# bottom = cm.get_cmap('Blues', 128)
# newcolors = np.vstack((top(np.linspace(0, 1, 128)),
#                        bottom(np.linspace(0, 1, 128))))
# newcmp = ListedColormap(newcolors, name='OrangeBlue')

# sn.heatmap(df_association, cmap = newcmp, vmin=-1, vmax=1)
# plt.xticks(rotation=50)
# plt.ion() #Reenable interactive mode just in case we might need it later

In [ ]:
print(df_tag.loc[genre][['Count','Median Rev','Median Price','Quartile']])
if genre2:
    print(df_tag.loc[genre2][['Count','Median Rev','Median Price','Quartile']])
print()
if not genre2:
    print("TOP "+genre+" games: ")
    print(df_bool[df_bool[genre] == True].sort_values('Estimated Rev',ascending=False)[['Name','Estimated Rev','Price']].head(n=25))
else:
    print("TOP "+genre+" + "+genre2+" games: ")
    print(df_bool[(df_bool[genre] == True)&(df_bool[genre2] == True)].sort_values('Estimated Rev',ascending=False)[['Name','Estimated Rev','Price']].head(n=25))

    


In [ ]:
#TODO: Was having trouble making the chart interactive so I got ChatGPT to rewrite this cell and he ended up using plotly instead - I think this is way more convenient and appears to be faster to load, I should consider replacing the other ones with plotly as well.

import plotly.graph_objs as go

# Set default figure size
plt.rcParams['figure.figsize'] = [25, 25]

# Create a trace for each genre
traces = []
tolerance_rev = 0 # Used to filter out the graph to make it more readable, 10k is a good value to use if you wish to see only the top median rev genres
c = 0
k = 0
for current_genre in tag_frequency.keys():
    if not (current_genre in tags_exclude):
        series_genre = df_bool[(df_bool[current_genre] == True)].sort_values('Release Date', ascending=True)
        series_genre = series_genre.groupby(series_genre['Release Date'].dt.year)['Estimated Rev'].median()
        c+=1
        if( series_genre.median() >= tolerance_rev ):
            k+=1
            trace = go.Scatter(x=series_genre.index, y=series_genre.values, name=current_genre, hovertext=current_genre)
            traces.append(trace)
            
# Set layout for the chart
layout = go.Layout(title='Revenue by Year', xaxis=dict(title='Year', range=[2016, 2022]), yaxis=dict(title='Estimated Revenue', range=[0, 5000000]), hovermode='closest')

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the chart
fig.show()



# Genre graph
traces = []
for current_genre in tag_frequency.keys():
    if not (current_genre in tags_exclude):
        series_genre = df_bool[(df_bool[current_genre] == True)&(df_bool[genre] == True)].sort_values('Release Date', ascending=True)
        series_genre = series_genre.groupby(series_genre['Release Date'].dt.year)['Estimated Rev'].median()
        trace = go.Scatter(x=series_genre.index, y=series_genre.values, name=current_genre, hovertext=current_genre)
        traces.append(trace)

# Set layout for the chart
layout = go.Layout(title='Revenue by Year', xaxis=dict(title='Year', range=[2016, 2022]), yaxis=dict(title='Estimated Revenue', range=[0, 5000000]), hovermode='closest')

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the chart
fig.show()


In [ ]:
# BOX PLOT of all tags, sorted by first quartile
# Note that some genre have very large confidence intervals
from numpy import median

plt.rcParams['figure.figsize'] = [65, 65]
fig,ax = plt.subplots()
list_box = list()
tick_labels = list()

plt.ioff() #Disable interactive mode - too much text

df_tag = df_tag.sort_values("Median Rev",ascending=False)
df_tag = df_tag.sort_values("Quartile",ascending=False)

for text in df_tag.index:
    if text is not 'CUSTOM':
        list_box.append( df_bool[df_bool[text]==True]['Estimated Rev'].to_list() )
        tick_labels.append(text)
    
ax.set_yticks(np.arange(0,1000000,10000))
#ax.axhline(y=20000)
plt.yscale("log")
print(df_tag.index.tolist()[0:50])
ax.boxplot(list_box,notch=True,showfliers=True,patch_artist=True,boxprops=dict(facecolor="red"))

ax.set_xticklabels(tick_labels,rotation=90,size=12)
plt.ion() #Reenable interactive mode just in case we might need it later


In [ ]:
# EXTRA: Pricing analysis, GENRE INDEPENDENT
import math

plt.rcParams['figure.figsize'] = [25, 25]
fig,ax = plt.subplots()
df_grouped = df_bool.copy()
df_grouped = df_grouped[['Price','Estimated Rev','Estimated Sales']]
round_number = 5
##Round so prices aren't X.99
#df_grouped['Price'] = df_grouped.apply(lambda x: round(x['Price']),axis=1)
#Round to nearest 'round_number'
df_grouped['Price'] = df_grouped.apply(lambda x: max(round(x['Price']/round_number)*round_number,1),axis=1)
df_grouped = df_grouped.groupby('Price')[['Estimated Rev','Estimated Sales']].median()

ax.set_xlim([0,60])
ax.set_ylim([0,40000])
ax.ticklabel_format(useOffset=False, style='plain')
ax.set_yticks(np.arange(0,df_grouped['Estimated Sales'].max()*60,df_grouped['Estimated Sales'].max()*60/10))
ax.set_xticks(np.arange(0,60,5))

ax.grid(axis="x")

ax.axhline(y=df_grouped.iloc[0]['Estimated Rev'],c="red")

# Theory
ax.plot(df_grouped.index.to_list(), (df_grouped['Estimated Sales'].to_numpy()*df_grouped.index.to_numpy()).tolist(),ls='--' )
# Reality
ax.plot(df_grouped.index.to_list(), df_grouped['Estimated Rev'] )